In [1]:
import numpy as np # linear algebra
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/hackathon/Surveydata_train (2).csv
/kaggle/input/hackathon/Traveldata_train (2).csv
/kaggle/input/shinkansen-travel-experience/Surveydata_test_(1).csv
/kaggle/input/shinkansen-travel-experience/Traveldata_train_(1)_(2).csv
/kaggle/input/shinkansen-travel-experience/Surveydata_train_(1)_(1).csv
/kaggle/input/shinkansen-travel-experience/Traveldata_test_(1).csv


In [2]:
data1 = pd.read_csv('/kaggle/input/shinkansen-travel-experience/Traveldata_train_(1)_(2).csv')
data1.head()
data2 = pd.read_csv('/kaggle/input/shinkansen-travel-experience/Surveydata_train_(1)_(1).csv')
data2.head()

data = pd.merge(data1, data2, on='ID', how='inner')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94379 entries, 0 to 94378
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       94379 non-null  int64  
 1   Gender                   94302 non-null  object 
 2   CustomerType             85428 non-null  object 
 3   Age                      94346 non-null  float64
 4   TypeTravel               85153 non-null  object 
 5   Travel_Class             94379 non-null  object 
 6   Travel_Distance          94379 non-null  int64  
 7   DepartureDelay_in_Mins   94322 non-null  float64
 8   ArrivalDelay_in_Mins     94022 non-null  float64
 9   Overall_Experience       94379 non-null  int64  
 10  Seat_comfort             94318 non-null  object 
 11  Seat_Class               94379 non-null  object 
 12  Arrival_time_convenient  85449 non-null  object 
 13  Catering                 85638 non-null  object 
 14  Platform_location     

In [4]:
data.isnull().sum()

ID                            0
Gender                       77
CustomerType               8951
Age                          33
TypeTravel                 9226
Travel_Class                  0
Travel_Distance               0
DepartureDelay_in_Mins       57
ArrivalDelay_in_Mins        357
Overall_Experience            0
Seat_comfort                 61
Seat_Class                    0
Arrival_time_convenient    8930
Catering                   8741
Platform_location            30
Onboardwifi_service          30
Onboard_entertainment        18
Online_support               91
Onlinebooking_Ease           73
Onboard_service            7601
Leg_room                     90
Baggage_handling            142
Checkin_service              77
Cleanliness                   6
Online_boarding               6
dtype: int64

In [5]:
#test

In [6]:
#data.dropna(inplace=True,axis=0)

In [7]:
data.head()

,ID,Gender,CustomerType,Age,TypeTravel,Travel_Class,Travel_Distance,DepartureDelay_in_Mins,ArrivalDelay_in_Mins,Overall_Experience,...,Onboardwifi_service,Onboard_entertainment,Online_support,Onlinebooking_Ease,Onboard_service,Leg_room,Baggage_handling,Checkin_service,Cleanliness,Online_boarding
0,98800001,Female,Loyal Customer,52.0,NaN,Business,272,0.0,5.0,0,...,good,need improvement,acceptable,need improvement,need improvement,acceptable,need improvement,good,need improvement,poor
1,98800002,Male,Loyal Customer,48.0,Personal Travel,Eco,2200,9.0,0.0,0,...,good,poor,good,good,excellent,need improvement,poor,need improvement,good,good
2,98800003,Female,Loyal Customer,43.0,Business travel,Business,1061,77.0,119.0,1,...,need improvement,good,excellent,excellent,excellent,excellent,excellent,good,excellent,excellent
3,98800004,Female,Loyal Customer,44.0,Business travel,Business,780,13.0,18.0,0,...,acceptable,need improvement,acceptable,acceptable,acceptable,acceptable,acceptable,good,acceptable,acceptable
4,98800005,Female,Loyal Customer,50.0,Business travel,Business,1981,0.0,0.0,1,...,need improvement,good,excellent,good,good,good,good,good,good,good


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94379 entries, 0 to 94378
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       94379 non-null  int64  
 1   Gender                   94302 non-null  object 
 2   CustomerType             85428 non-null  object 
 3   Age                      94346 non-null  float64
 4   TypeTravel               85153 non-null  object 
 5   Travel_Class             94379 non-null  object 
 6   Travel_Distance          94379 non-null  int64  
 7   DepartureDelay_in_Mins   94322 non-null  float64
 8   ArrivalDelay_in_Mins     94022 non-null  float64
 9   Overall_Experience       94379 non-null  int64  
 10  Seat_comfort             94318 non-null  object 
 11  Seat_Class               94379 non-null  object 
 12  Arrival_time_convenient  85449 non-null  object 
 13  Catering                 85638 non-null  object 
 14  Platform_location     

In [9]:
data['Overall_Experience'].value_counts()

Overall_Experience
1    51593
0    42786
Name: count, dtype: int64

In [10]:
data.drop(['ID'], inplace=True, axis=1)

In [11]:
from sklearn.impute import SimpleImputer

# Numerical columns
numerical_features = data.select_dtypes(include=['float64', 'int64']).columns
numerical_imputer = SimpleImputer(strategy='mean')
data[numerical_features] = numerical_imputer.fit_transform(data[numerical_features])

# Categorical columns
categorical_features = data.select_dtypes(include=['object']).columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
data[categorical_features] = categorical_imputer.fit_transform(data[categorical_features])

In [12]:
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

columns_to_remove_outliers = ['Travel_Distance', 'DepartureDelay_in_Mins','ArrivalDelay_in_Mins']

for column in columns_to_remove_outliers:
    data = remove_outliers_iqr(data, column)

In [13]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each column with object (categorical) dtype
for column in data.select_dtypes(include=['object']).columns:
    data[column] = label_encoder.fit_transform(data[column])

# Display the first few rows of the updated DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68615 entries, 0 to 94377
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Gender                   68615 non-null  int64  
 1   CustomerType             68615 non-null  int64  
 2   Age                      68615 non-null  float64
 3   TypeTravel               68615 non-null  int64  
 4   Travel_Class             68615 non-null  int64  
 5   Travel_Distance          68615 non-null  float64
 6   DepartureDelay_in_Mins   68615 non-null  float64
 7   ArrivalDelay_in_Mins     68615 non-null  float64
 8   Overall_Experience       68615 non-null  float64
 9   Seat_comfort             68615 non-null  int64  
 10  Seat_Class               68615 non-null  int64  
 11  Arrival_time_convenient  68615 non-null  int64  
 12  Catering                 68615 non-null  int64  
 13  Platform_location        68615 non-null  int64  
 14  Onboardwifi_service      68

In [14]:
data['Overall_Experience'].value_counts()

Overall_Experience
1.0    39223
0.0    29392
Name: count, dtype: int64

In [15]:
#test

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Separate features and target variable
X = data.drop(columns=['Overall_Experience'])  # Replace 'target_column' with the name of your target column
y = data['Overall_Experience']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Random Forest Model
rf_model = RandomForestClassifier(n_estimators=200, criterion='entropy', random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

# Evaluate Random Forest Model
print("Random Forest Accuracy:", accuracy_score(y_test, rf_predictions))
print("Random Forest Classification Report:\n", classification_report(y_test, rf_predictions))

# XGBoost Model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

# Evaluate XGBoost Model
print("XGBoost Accuracy:", accuracy_score(y_test, xgb_predictions))
print("XGBoost Classification Report:\n", classification_report(y_test, xgb_predictions))


Random Forest Accuracy: 0.9464402827370109
Random Forest Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      0.94      0.94      5877
         1.0       0.96      0.95      0.95      7846

    accuracy                           0.95     13723
   macro avg       0.94      0.95      0.95     13723
weighted avg       0.95      0.95      0.95     13723

XGBoost Accuracy: 0.950375282372659
XGBoost Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.95      0.94      5877
         1.0       0.96      0.95      0.96      7846

    accuracy                           0.95     13723
   macro avg       0.95      0.95      0.95     13723
weighted avg       0.95      0.95      0.95     13723



In [17]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu', kernel_regularizer=l2(0.001)))  # L2 regularization
model.add(Dropout(0.5))  # Dropout layer with 50% dropout rate
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print("Test Accuracy:", accuracy)

# Make predictions
predictions = model.predict(X_test)
predicted_classes = (predictions > 0.5).astype("int32")  # Convert probabilities to binary predictions (1 or 0)


/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


I0000 00:00:1731658231.769162      67 service.cc:145] XLA service 0x7e92ec00c270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731658231.769216      67 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


  99/1373 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5409 - loss: 0.8849

I0000 00:00:1731658236.392428      67 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1373/1373 ━━━━━━━━━━━━━━━━━━━━ 13s 5ms/step - accuracy: 0.6914 - loss: 0.7147 - val_accuracy: 0.8514 - val_loss: 0.4330
Epoch 2/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8291 - loss: 0.4648 - val_accuracy: 0.8905 - val_loss: 0.3306
Epoch 3/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8692 - loss: 0.3691 - val_accuracy: 0.9087 - val_loss: 0.2854
Epoch 4/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8844 - loss: 0.3355 - val_accuracy: 0.9104 - val_loss: 0.2689
Epoch 5/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8903 - loss: 0.3197 - val_accuracy: 0.9126 - val_loss: 0.2618
Epoch 6/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8944 - loss: 0.3082 - val_accuracy: 0.9160 - val_loss: 0.2501
Epoch 7/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8964 - loss: 0.3021 - val_accuracy: 0.9157 - val_loss: 0.2489
Epoch 8/50
1373/1373 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8957 - loss: 0.3005 - val_accura

In [18]:
pip install tensorflow-decision-forests

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [19]:
import pandas as pd
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split

# Assuming 'data' is your DataFrame
#X = data.drop(columns=['Overall_Experience'])  # Features (remove the target column)
#y = data['Overall_Experience']  # Target variable (Overall_Experience)

# Split the data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Combine features and target into a single dataframe for train and test datasets
train_data_combined = X_train.copy()
train_data_combined['Overall_Experience'] = y_train

test_data_combined = X_test.copy()
test_data_combined['Overall_Experience'] = y_test

# Convert pandas DataFrame to TensorFlow dataset
train_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(train_data_combined, task=tfdf.keras.Task.CLASSIFICATION, label="Overall_Experience")
test_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(test_data_combined, task=tfdf.keras.Task.CLASSIFICATION, label="Overall_Experience")

# Create the model
model = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.CLASSIFICATION)

# Train the model
model.fit(train_dataset)

# Evaluate the model
evaluation = model.evaluate(test_dataset)
print("Evaluation result:", evaluation)

# Model summary and feature importances
model.summary()


Use /tmp/tmpys4l6j88 as temporary training directory
Reading training dataset...
Training dataset read in 0:00:06.989133. Found 54892 examples.
Training model...


[INFO 24-11-15 08:13:23.5379 UTC kernel.cc:1233] Loading model from path /tmp/tmpys4l6j88/model/ with prefix 821771525e8343d5
[INFO 24-11-15 08:13:26.7048 UTC decision_forest.cc:734] Model loaded with 300 root(s), 863302 node(s), and 23 input feature(s).
[INFO 24-11-15 08:13:26.7048 UTC abstract_model.cc:1362] Engine "RandomForestOptPred" built
[INFO 24-11-15 08:13:26.7049 UTC kernel.cc:1061] Use fast generic engine


Model trained in 0:00:19.415268
Compiling model...
Model compiled.
14/14 [==============================] - 4s 44ms/step - loss: 0.0000e+00
Evaluation result: 0.0
Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (23):
	Age
	ArrivalDelay_in_Mins
	Arrival_time_convenient
	Baggage_handling
	Catering
	Checkin_service
	Cleanliness
	CustomerType
	DepartureDelay_in_Mins
	Gender
	Leg_room
	Onboard_entertainment
	Onboard_service
	Onboardwifi_service
	Online_boarding
	Online_support
	Onlinebooking_Ease
	Platform_location
	Seat_Class
	Seat_comfort
	Travel_Class
	Travel_Distance
	TypeTravel

No weights

Variable Importance: INV_MEAN_MIN_DEPTH:
    1.   "